In [1]:
# Importing necessary libraries
from transformers import LayoutLMForTokenClassification, LayoutLMTokenizerFast
from transformers import TrainingArguments, Trainer

/Users/zwang/myenv/lib/python3.9/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/zwang/myenv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/zwang/myenv/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Loading the pre-trained LayoutLM model and tokenizer
model = LayoutLMForTokenClassification.from_pretrained("microsoft/layoutlm-base-uncased")
tokenizer = LayoutLMTokenizerFast.from_pretrained("microsoft/layoutlm-base-uncased")


config.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/453M [00:00<?, ?B/s]

Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/170 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [7]:
# data 'project-4-at-2023-11-28-20-38-6ed7fdd6.json'
from datasets import load_dataset

In [9]:
your_train_dataset = load_dataset('json', data_files='project-4-at-2023-11-28-20-38-6ed7fdd6.json')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [10]:
your_eval_dataset = load_dataset('json', data_files='project-4-at-2023-11-28-20-38-6ed7fdd6.json')

In [12]:
from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer)

In [13]:
# Setting up the training arguments for the Trainer
training_args = TrainingArguments(
    output_dir="./results",  # Directory to save the training results
    num_train_epochs=3,  # Number of training epochs
    per_device_train_batch_size=8,  # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    warmup_steps=500,  # Number of warmup steps
    weight_decay=0.01,  # Weight decay coefficient
    logging_dir="./logs",  # Directory to save logs
)

# Creating the Trainer object
trainer = Trainer(
    model=model,  # The LayoutLM model
    args=training_args,  # Training arguments
    train_dataset=your_train_dataset,  # Training dataset
    eval_dataset=your_eval_dataset,  # Evaluation dataset
    data_collator=data_collator,  # Data collator
)

In [14]:
# Training the model
trainer.train()

  0%|          | 0/3 [00:00<?, ?it/s]

KeyError: "Invalid key: 0. Please first select a split. For example: `my_dataset_dictionary['train'][0]`. Available splits: ['train']"